In [1]:
%load_ext autoreload
%autoreload 2

In [5]:
import os
import torch
from visualize._utils import draw_connectome
from utils import ROOT_DIR
from torch_geometric.data import Data
from tests.leandro.plots import *
from data._main import get_dataset
from omegaconf import OmegaConf

In [6]:
# load the connectome data
graph_tensors = torch.load(
    os.path.join(ROOT_DIR, "data", "processed", "connectome", "graph_tensors.pt")
)

# make the graph
graph = Data(**graph_tensors)

print('Graph attributes: {}'.format(graph.keys))
print('Num. nodes: {}, Num. edges: {}, Num. node features: {}'.format(graph.num_nodes, graph.num_edges, graph.num_node_features))

Graph attributes: ['num_classes', 'x', 'edge_index', 'pos', 'edge_attr', 'idx_to_neuron', 'n_id', 'y', 'node_type']
Num. nodes: 302, Num. edges: 4396, Num. node features: 1024


In [15]:
neuron_to_idx = {u: v for (v, u) in graph.idx_to_neuron.items()}
idx_to_neuron = {v: u for (v, u) in graph.idx_to_neuron.items()}

In [23]:
def get_connected_neurons(graph, neuron_name, index=False):

    neuron_to_idx = {u: v for (v, u) in graph.idx_to_neuron.items()}

    # Get the edge indices corresponding to the given node
    node_index = neuron_to_idx[neuron_name]
    connected_edge_indices = (graph.edge_index[0] == node_index).nonzero()

    # Get the connected nodes by extracting the second row of the connected_edge_indices
    connected_nodes = graph.edge_index[1, connected_edge_indices].flatten()
    
    if index == True:
        return connected_nodes.detach().numpy()
    else:
        return np.array([idx_to_neuron[idx] for idx in connected_nodes.detach().numpy()])

print(graph.edge_index.shape) # Row 0: source, Row 1: target

print(graph.edge_attr.shape) # Column 0: gap junction connections, Column 1: chemical synapse connections

torch.Size([2, 4396])
torch.Size([4396, 2])


In [27]:
get_connected_neurons(graph, 'M3L', index=False)

array(['NSML', 'NSMR'], dtype='<U4')

In [19]:
config = OmegaConf.load("/home/lrvnc/Projects/worm-graph/conf/dataset.yaml")
flavell_dataset = get_dataset(config)

Chosen dataset(s): ['Flavell2023']
Num. worms: 50

